# Causal Search on the BRAIN AI Summarized Data

## Imports

In [3]:
# Causal Discovery
## Constraint based
from causallearn.search.ConstraintBased.FCI import fci
from causallearn.search.ConstraintBased.PC import pc
## Permutation based
from causallearn.search.PermutationBased.GRaSP import grasp
## Score based
from causallearn.search.ScoreBased.GES import ges
## Background knowledge
from causallearn.utils.PCUtils import BackgroundKnowledge
from causallearn.graph.GraphNode import GraphNode

# Data wrangling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Graph drawing
import networkx as nx

## Importing and Cleaning the Summarized data

In [4]:
# Importing the Summarized Data (Use this file since it was re-run after fixing the issues with Lorazepam in the biomrks.csv table)
df12 = pd.read_csv('Modified Code/Approaches/CT-MRI-EEG-BH outcome_12 censor horizon_48 time window/datasets/summary_df.csv')

# 608 columns is too much for our current environment to run. Furthermore, it increases the risk of colinearity in our data.
# Colinearity will prevent us from using fisher's Z-test as a conditional independence metric
# We need to find a way to reduce the number of columns.

In [5]:
## Eddie Functions ##
from pandas.api.types import is_numeric_dtype

def avg_ranges(values):
    """
    Purpose: Get the average of a discretized range that is in strings
    Inputs:  A discretized feature column with an interval represented as a string
    Outputs:  The average of the range as a numeric.
    """
    for i in range(len(values)):
        if values[i] == 'missing':
            values[i] = np.nan
        elif '_to_' in values[i]:
            item = values[i].replace('neg', '-').replace('_to_', ', ')
            item  = np.fromstring(item, sep = ', ', count = 2)
            values[i] = np.mean(item)
        elif 'Month' in values[i]:
            item = values[i].replace('Month', '').replace('Over', '').replace('Months', '').replace('to', ' ').replace('Under', '')
            item = np.fromstring(item, sep = 'to')
            values[i] = np.mean(item)
        else:
            values[i] = values[i]
    return values 



def categorize(values):
    """
    Purpose: Create ordered numeric categories of the discretized features
    Inputs:  A discretized feature column with an interval represented as a string. The format must be a pandas series.
    Outputs: Ordered categorical features as a pandas series.
    """
    x = values.astype(str)
    if 'arrive_dtm' in values.name:
        return values
    elif 'missingness' in values.name:
        to_return = pd.Categorical(values.values, ordered= True, categories = [0, 1])
        to_return = pd.Series(to_return, index = values.index)
        return to_return
    elif 'Pupillary' in values.name:
        to_return = pd.Categorical(x.values, ordered = True, categories = ['normal', 'one sluggish', 'both sluggish', 'one nonreactive',  'both nonreactive']).codes
        to_return = pd.Series(to_return, index = values.index)
        return to_return
    elif 'Ventilator' in values.name:
        to_return = pd.Categorical(x.values, ordered = True, categories = ['target', 'other']).codes
        to_return = pd.Series(to_return, index = values.index)
        return to_return
    elif x.apply(lambda x: 'All' in x).any():
        to_return = pd.Categorical(x.values, ordered = True, categories = ['All'])
        to_return = pd.Series(to_return, index = values.index)
        return to_return
    elif not x.apply(lambda x: 'to' in x).any():
        return values
    else:
        return pd.Series(pd.Categorical(avg_ranges(x.values), ordered = True), index = values.index)

In [6]:
df12 = df12.apply(categorize)

C:\Users\EDP37\AppData\Local\Temp\ipykernel_5316\1398877953.py:19: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  item = np.fromstring(item, sep = 'to')


### For each biomarker, keep the feature that best correlates with the outcome

In [7]:
# We will use the Biomarker column to filter out columns
biomrkrs = pd.read_csv('./biomarker_attr.csv')

k = 1 # The number of features per biomarker with the top k correlations with the outcome
topk = [] # keeping track of the top feature

# Search for the features with the highest correlation with the outcome per biomarker
for biomarker in biomrkrs.Biomarker:
    try: # if it fails, continue an keep track of what failed
        to_append = df12.filter(like=biomarker).corrwith(df12.outcome).sort_values(ascending=False)[:k].index.values
        topk.append(to_append)
    except:
        print(biomarker) # it seems like it is only the categorical biomarkers that are failing

# Don't forget to add the outcome 
topk.append(np.array(['outcome', 'age_group']))
topk = np.concatenate(topk)

# Keep only the features in the topk list
df12 = df12.filter(topk)


In [6]:
df12.to_csv('12hour.csv', index=False, na_rep ='*')
df12.head()

,Base deficit first,Base excess range max,Bicarbonate first,Blood urea nitrogen first,Chloride slope first,Cisatracurium,C-Reactive Protein slope first,Creatinine pct change first,CRRT Therapy Type,DBP range max,...,SBP slope min,Sodium range min,SpO2 median,Temperature range min,Ventilated,Ventilator Make/Model mode,Weight pct change first,White blood cell count seclast,outcome,age_group
0,NaN,NaN,23.0,7.0,-0.000756,0.0,NaN,-50.0,0.0,11.0,...,0.019171,2.0,97.0,1.0,0.0,-1,NaN,NaN,0.0,1.0
1,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,28.0,...,0.006519,0.0,97.0,1.6,0.0,-1,NaN,19.3,0.0,12.0
2,NaN,4.0,34.5,NaN,NaN,0.0,NaN,NaN,0.0,22.0,...,0.013454,1.0,97.0,0.6,1.0,0,NaN,12.6,0.0,12.0
3,NaN,NaN,18.0,NaN,NaN,0.0,NaN,NaN,0.0,42.0,...,NaN,0.0,96.0,0.2,0.0,-1,NaN,10.8,1.0,144.0
4,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,25.0,...,0.016188,NaN,98.0,0.4,1.0,1,NaN,NaN,0.0,12.0


In [8]:
labs = biomrkrs.Biomarker.to_list()
labs.append('Neurologic Morbidity')
labs.append('Age')

## Causal Discovery

In [10]:
# We will run 2 causal discovery algorithms: FCI, PC
# I will use the G-squared as the independence test since the others seem to fail probably due to the NANs in our data
# These runs won't use background knowledge
bk = BackgroundKnowledge.BackgroundKnowledge()

bk.add_forbidden_by_pattern('outcome', 'age_group')
bk.add_forbidden_by_pattern('Neurologic Morbidity', 'Age')

for biomarker in df12.iloc[:,:-2].columns.values:
    bk.add_forbidden_by_pattern('outcome', biomarker)

    bk.add_forbidden_by_pattern('age_group', biomarker)

for biomarker in labs[:-2]:
        bk.add_forbidden_by_pattern('Neurologic Morbidity', biomarker)
        bk.add_forbidden_by_pattern('Age', biomarker)
                            



In [11]:
#PC
g_pc = pc(df12.to_numpy(), alpha=0.05,  node_names=labs, indep_test='gsq', background_knowledge = bk)

# visualization
from causallearn.utils.GraphUtils import GraphUtils
graph = GraphUtils.to_pydot(g_pc.G, labels = labs, dpi=600)
# Get a list of nodes and edges
nodes = graph.get_nodes()
edges = graph.get_edges()

# Find all nodes that are part of edges (connected nodes)
connected_nodes = set()
for edge in edges:
    connected_nodes.add(edge.get_source())
    connected_nodes.add(edge.get_destination())


# Remove nodes without edges, preserving their attributes
for node in nodes:
    node_name = int(node.get_name().strip('"'))  # Strip quotes around names
    if node_name not in connected_nodes:
        graph.del_node(node)
# # The graph now only contains nodes with edges, and their labels are preserved
# print(graph.to_string())  # Output will show the graph without isolated nodes but with labels intact

# The graph now only contains nodes with edges
# print(graph.to_string())  # Output shows graph structure without isolated nodes
graph.set_graph_defaults(dpi=1000)
graph.set_size('"90,90"') 
graph.set_graph_defaults(fontsize="40")
graph.write_png('pc_12hour.png')

  0%|          | 0/47 [00:00<?, ?it/s]

In [12]:
#FCI
g_fci, edges_fci = fci(df12.to_numpy(), alpha=0.05, node_names=labs, independence_test_method='gsq', background_knowledge = bk)


# visualization
from causallearn.utils.GraphUtils import GraphUtils
graph = GraphUtils.to_pydot(g_fci, edges_fci, labels=labs, dpi=600)
# Get a list of nodes and edges
nodes = graph.get_nodes()
edges = graph.get_edges()

# Find all nodes that are part of edges (connected nodes)
connected_nodes = set()
for edge in edges:
    connected_nodes.add(edge.get_source())
    connected_nodes.add(edge.get_destination())


# Remove nodes without edges, preserving their attributes
for node in nodes:
    node_name = int(node.get_name().strip('"'))  # Strip quotes around names
    if node_name not in connected_nodes:
        graph.del_node(node)
# # The graph now only contains nodes with edges, and their labels are preserved
# print(graph.to_string())  # Output will show the graph without isolated nodes but with labels intact

# The graph now only contains nodes with edges
# print(graph.to_string())  # Output shows graph structure without isolated nodes
graph.set_graph_defaults(dpi=1000)
graph.set_size('"90,90"') 
graph.set_graph_defaults(fontsize="40")
graph.write_png('fci_12hour.png')

  0%|          | 0/47 [00:00<?, ?it/s]

Starting BK Orientation.
Finishing BK Orientation.
Starting BK Orientation.
Finishing BK Orientation.
X14 --> X16
X14 --> X22
X14 --> X24
X14 --> X43
X16 --> X24
X21 --> X46
